# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2025-04-29 14:21:53,002 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

In [6]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-04-29 14:27:52,895 | INFO     |:  | Number of observations per grid cell: 18
2025-04-29 14:27:52,901 | INFO     |:  | Precaching Views...
2025-04-29 14:27:55,362 | INFO     |:  | Precaching Time: 2.460 secs
2025-04-29 14:27:55,370 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-29 14:27:55,372 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [7]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-29 14:28:04,988 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-29 14:28:04,990 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2025-04-29 14:28:07,954 | INFO     |: Update Step
2025-04-29 14:28:07,964 | INFO     |:      | Update Time: 0.004 secs
2025-04-29 14:28:07,966 | INFO     |: ---------- UPDATE STATS -----------
2025-04-29 14:28:07,989 | INFO     |: GT index      : (6, 4, 9)
2025-04-29 14:28:07,991 | INFO     |: Bel index     : (np.int64(5), np.int64(4), np.int64(9)) with prob = 0.9940916
2025-04-29 14:28:07,992 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-04-29 14:28:07,993 | INFO     |: GT            : (0.000, 0.000, 360.000)
2025-04-29 14:28:07,994 | INFO     |: Belief        : (0.000, 0.000, 10.000)
2025-04-29 14:28:07,995 | INFO     |: POS ERROR     : (-0.000, -0.000, 350.000)
2025-04-29 14:28:07,995 | INFO     |: ---------- UPDATE STATS -----------


In [8]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2025-04-29 14:28:16,094 | INFO     |: Prediction Step
2025-04-29 14:28:16,148 | INFO     |:  | Prediction Time: 0.050 secs
2025-04-29 14:28:16,151 | INFO     |: ---------- PREDICTION STATS -----------
2025-04-29 14:28:16,163 | INFO     |: GT index         : (6, 3, 6)
2025-04-29 14:28:16,171 | INFO     |: Prior Bel index  : (np.int64(6), np.int64(4), np.int64(7)) with prob = 0.1048422
2025-04-29 14:28:16,173 | INFO     |: POS ERROR        : (-0.013, -0.090, -10.107)
2025-04-29 14:28:16,177 | INFO     |: ---------- PREDICTION STATS -----------
2025-04-29 14:28:19,109 | INFO     |: Update Step
2025-04-29 14:28:19,113 | INFO     |:      | Update Time: 0.003 secs
2025-04-29 14:28:19,114 | INFO     |: ---------- UPDATE STATS -----------
2025-04-29 14:28:19,133 | INFO     |: GT index      : (6, 3, 6)
2025-04-29 14:28:19,148 | INFO     |: Bel index     : (np.int64(6), np.int64(4), np.int64(6)) with prob = 1.0
2025-04-29 14:28:19,151 | INFO     |: Bel_bar